# Django ORM - Instrukcje DML w relacjach

W bazach relacyjnych tabele mogą być powiązane ze sobą relacjami. W zależności od natury tego powiązania wyróżniamy kilka typów relacji. W Django ORM możemy wyróżnić trzy podstawowe typy relacji: jeden-do-jednego, jeden-do-wielu i wiele-do-wielu. Każda z tych relacji posiada w Django ORM swój odpowiednik w postaci pola modelu. I tak:
* **OneToOneField** to pole odpowiadające relacji jeden do jednego
* **ForeignKey** to pole odpowiadające relacji jeden do wielu
* **ManyToManyField** to pole odpowiadające relacji wiele do wielu

Omówmy je po kolei.

## OneToOneField

Relacja jeden do jednego występuje w przypadku kiedy rekord jednej tabeli może być powiązany z jednym i tylko jednym wpisem drugiej tabeli. Przykładem takiej relacji może być tabela stolica oraz tabela państwo. Warszawa jest stolicą tylko jednego państwa - Polski, Polska ma przypisaną tylko jedną stolicę - Warszawę.
"Mówimy Warszawa myślimy Polska, mówimy Polska myślimy Warszawa". Relacja jest symetryczna dlatego nie ma znaczenia, w której z tabelek (stolica, czy państwo) umieścimy kolumnę dla tej relacji.

**Definicje modeli**

W modelach mamy dwie klasy: Country i Capitol. Relacje OneToOneField umieściliśmy po stronie modelu Country. Atrybut przechowujący relację nazwaliśmy capitol.

<code>class Country(models.Model):
    name = models.CharField(max_length=64)
    capitol = models.OneToOneField('Capitol', on_delete=models.CASCADE)
</code>

<code>class Capitol(models.Model):
    name = models.CharField(max_length=64)
</code>

### C z CRUD

In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [8]:
# 1. Tworzymy wpis w tabeli Capitol
warsaw = Capitol.objects.create(name="Warsaw")

In [9]:
# 2. Tworzymy wpis tabeli Country
django = Country.objects.create(name="Poland")

IntegrityError: NOT NULL constraint failed: taskapp_country.capitol_id

In [10]:
# Nie możemy! Dostaliśmy IntegrityError.
# Dlaczego?
# Ponieważ pole capitol (z referencją do modelu Capitol) nie może być puste, a my próbując stworzyć 
# wpis nie podaliśmy wartości w tym polu.

# Spróbujmy dopisać
django = Country.objects.create(name="Poland", capitol="Warsaw")

ValueError: Cannot assign "'Warsaw'": "Country.capitol" must be a "Capitol" instance.

In [11]:
# Znów się nie udało. Tym razem wprowadziliśmy wartość, ale była ona typu string. 
# Python informuje nas, że wprowadzana w tym polu wartość powinna być instancją klasy 
# (modelu) Capitol.

# Ale my już mamy taką jedną instancję. Przypisaliśmy ją do zmiennej warsaw, tworząc 
# pierwszy wpis w tabeli Captiol.
warsaw

<Capitol: Capitol object (5)>

In [13]:
# czyli wystarczy teraz ją wykorzystać. To pierwszy sposób tworzenia.

# Metada I (instancja modelu)
poland = Country.objects.create(name="Poland", capitol=warsaw)

In [18]:
# Oczywiście możemy używać tu również dwóch pozostałych metod tworzenia wpisów, 
# które już poznaliśmy.

# Tworzymy wpis dla Francji
paris = Capitol(name="Paris")
paris.save()
france = Country(name="France", capitol=paris)
france.save()

# Tworzymy wpis dla Włoch
rome = Capitol()
rome.name = "Rome"
rome.save()
italy = Country()
italy.name = "Italy"
italy.capitol = rome
italy.save()

In [20]:
# Ale mamy jeszcze inny sposób. 

# Popatrzmy na to co posiada tabelka Country
print(dir(Country))

['DoesNotExist', 'MultipleObjectsReturned', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_check_column_name_clashes', '_check_constraints', '_check_default_pk', '_check_field_name_clashes', '_check_fields', '_check_id_field', '_check_index_together', '_check_indexes', '_check_local_fields', '_check_long_column_names', '_check_m2m_through_same_relationship', '_check_managers', '_check_model', '_check_model_name_db_lookup_clashes', '_check_ordering', '_check_property_name_related_field_accessor_clashes', '_check_single_primary_key', '_check_swappable', '_check_unique_together', '_do_insert', '_do_update', '_get_FIELD_display', '_get_expr_references', '_get_next_

In [21]:
# Widać, że Django poza polem capitol wygenerowało również dla modelu Country pole capitol_id 
# reprezentujące id wpisu tabeli Capitol. Możemy użyć tego pola do utworzenia relacji.

# Metoda II (id instancji modelu)

# 1. Tworzymy wpis w tabeli Capitol
berlin = Capitol.objects.create(name="Berlin")
berlin_id = berlin.id  # -> 2

# 2. Tworzymy wpis w tabeli Country przekazując id instancji modelu Capitol
germany = Country.objects.create(name="Germany", capitol_id=berlin_id)

## ForeignKey

Relacja jeden do wielu jest najczęściej wykorzystywanym typem relacji. Występuje wtedy, kiedy wpis z jednej tabeli (tzw. tabeli rodzica) może być powiązany z wieloma wpisami z drugiej (tzw. tabeli dziecka), ale wpis z drugiej tabeli (tabeli dziecka) nie może być powiązany z wielom wpisami z pierwszej (tabeli rodzica). Innymi słowy rodzic może mieć wiele dzieci, ale dziecko może mieć tylko jednego rodzica. Przykładem takiej relacji może być tabela "miasto" oraz tabela "państwo". Gdańsk należy do Polski. Kraków należy do Polski. Mówiąc Gdańsk myślimy Polska (to strona relacja "jeden"). Podobnie mówiąc Kraków myślimy Polska. Ale już mówiąc Polska myślimy Gdańsk, Kraków, Wrocław, ... (to strona relacji "wiele"). 

Najprościej jest wyobrazić sobie relacje jeden-do-wielu jako strukturę hierarchiczną, czyli drzewo. Na górze mamy rodzica, a pod nim wiele dzieci. Tutaj Polska jest rodzicem, a Gdańsk, Kraków, Wrocław, ... dziećmi. To po której stronie umieścimy pole do przechowywania relacji zależy wyłącznie od nas. Zazwyczaj znacznie łatwiej myśli się o takiej relacji, kiedy pole umieści się po stronie dziecka (ponieważ dziecko ma tylko jednego rodzica). Czyli umieszczamy pole ForeignKey w modelu "miasto" i wpisy Gdańsk, Kraków, Wrocław posiadają referencje do wpisu Polska z modelu "państwo". 

Innym przykładem takiej relacji może być język programowania i framework. Jezyk programowania to np. Python. Framework to np. Django, Flask, Bottle. Rodzicem jest tu język programowania, dziećmi poszczególne frameworki (mówimy Python myślimy Django, Flask, Bottle... ale mówimy Django myślimy Python). Czyli pole z relacją najlepiej umieścić po stronie modelu Framework.

**Definicje modeli**

W modelach mamy dwie klasy: Language i Framework. Relacje ForignKey umieściliśmy po stronie modelu Framework (dziecko). Atrybut przechowujący relację nazwaliśmy language.

<code>class Language(models.Model):
    name = models.CharField(max_length=64)
</code>
<code>
    def __str__(self):
        return f"{self.name}"
</code>

<code>class Framework(models.Model):
    name = models.CharField(max_length=64)
    language = models.ForeignKey('Language', on_delete=models.CASCADE)
</code>
<code>
    def __str__(self):
        return f"{self.name} ({self.language})"
</code>

### C z CRUD

Operacje "C" niczym się nie różnią od tych dla pola OneToOneField. W ramach utrwalenia przypomnijmy, że mamy dwie metody.

In [16]:
# Metad I (instancja modelu)

python = Language.objects.create(name="python")
django = Framework.objects.create(name="django", language=python)

In [22]:
# Metad II (id instancji modelu)

java = Language.objects.create(name="java")
spring = Framework.objects.create(name="spring", language_id=java.id)

### R z CRUD

Na początek wyświetlmy wszystkie framework-i.

In [3]:
Framework.objects.all()  # doróbmy __str__ modelom Framework i Language

<QuerySet [<Framework: django (python)>, <Framework: Flask (python)>, <Framework: Bottle (python)>, <Framework: spring (java)>]>

In [ ]:
# A frameworki tylko dla konkretnego języka?
# Klasyczny filtr po polu language.

# 1. Pobieramy obiekt, po którym będziemy wyszukiwać.
python = Language.objects.get(name='python')

# 2. Wyszukujemy po pobranym obiekcie.
frameworks = Framework.objects.

## ManyToMany

Ostatnia z omawianych relacji to relacja wiele-do-wielu. Jest to najbardziej złożony typ relacji. Dotyczy sytuacji kiedy wpisy z jednej tabeli mogą być powiązane z wielom wpisami z drugiej oraz wpisy z drugiej tabeli mogą być powiązane z wieloma wpisami z tabeli pierwszej. Przykładem takiej relacji może być tabela Aktor oraz tabela Film. Mówimy Al Pacino myślimy Scareface, Gorączka, Ojciec Chrzestny... Mówimy Ojciec Chrzestny myślimy Al Pacino, Robert DeNiro, Marlon Brando ... Relację możemy umieścić w dowolnej z powiązanych tabeli.

**Definicje modeli**

W modelach mamy dwie klasy: Actor i Movie. Relacje ManyToMany umieszczamy po stronie Actor (ale równie dobrze moglibyśmy umieścić po stronie Movie). Atrybut przechowujący relację nazwaliśmy movies (zwróć uwagę na to, że tym razem w nazwie pola użyliśmy liczby mnogiej).

<code>class Actor(models.Model):
    name = models.CharField(max_length=64)
    movies = models.ManyToManyField('Movie')
</code>

<code>class Movie(models.Model):
    title = models.CharField(max_length=128)
</code>
